In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

from langchain.embeddings import HuggingFaceEmbeddings
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

True

## Wikipedia Retriever

In [3]:
from langchain_community.retrievers import WikipediaRetriever

In [4]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [8]:

# Define your query
query = "Who is the best batsman in the world?"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [9]:
docs

[Document(metadata={'title': 'Batting order (cricket)', 'summary': "In cricket, the batting order is the sequence in which batters play through their team's innings, there always being two batters taking part at any one time. All eleven players in a team are required to bat if the innings is completed (i.e., if the innings does not close early due to a declaration or other factor).\nThe batting order is colloquially subdivided into:\n\nOpeners (batters one and two)\nTop order (batters one to three)\nMiddle order (batters four to seven)\nLower order or Tailenders (batters eight to eleven)\nThe order in which the eleven players will bat is usually established before the start of a cricket match, but may be altered during play. The decision is based on factors such as each player's specialities; the position each batter is most comfortable with; each player's skills and attributes as a batter; possible combinations with other batters; and the match situation whereby, for example, the team

In [10]:
# Print retrieved content
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content}...")  # truncate for display


--- Result 1 ---
Content:
In cricket, the batting order is the sequence in which batters play through their team's innings, there always being two batters taking part at any one time. All eleven players in a team are required to bat if the innings is completed (i.e., if the innings does not close early due to a declaration or other factor).
The batting order is colloquially subdivided into:

Openers (batters one and two)
Top order (batters one to three)
Middle order (batters four to seven)
Lower order or Tailenders (batters eight to eleven)
The order in which the eleven players will bat is usually established before the start of a cricket match, but may be altered during play. The decision is based on factors such as each player's specialities; the position each batter is most comfortable with; each player's skills and attributes as a batter; possible combinations with other batters; and the match situation whereby, for example, the team may require a more defensive or attacking playe

## Vector Store Retriever

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

from langchain_core.documents import Document

In [12]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [13]:
# Step 2: Initialize embedding model
# embedding_model = OpenAIEmbeddings()
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [14]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [ ]:
query = "What is Chroma used for?"
results = retriever.invoke(query) 

In [16]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


In [17]:
results = vectorstore.similarity_search(query, k=2)

In [18]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


Note we can use either reviter or vector store to get similar text but with the help retriver we can change the stratergy of search and retriver is a runnable object we can use it to integrate with the help of chains

## MMR

In [19]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [20]:
from langchain_community.vectorstores import FAISS

# Initialize OpenAI embeddings
# embedding_model = OpenAIEmbeddings()
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [27]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0}  # k = top results, lambda_mult = relevance-diversity balance
)

In [28]:
query = "What is langchain?"
results = retriever.invoke(query)

In [29]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
MMR helps you get diverse results when doing similarity search.

--- Result 3 ---
Embeddings are vector representations of text.


## Multiquery Retriever

In [30]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [31]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [32]:
# Initialize OpenAI embeddings
# embedding_model = OpenAIEmbeddings()
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [41]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [35]:
from langchain_groq import ChatGroq

model = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.5 ## this is creative parameter
    )

In [45]:
#MultiQueryRetriever
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    llm=model
)

In [42]:
# Query
query = "How to improve energy levels and maintain balance?"

In [43]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [44]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.
******************************************************************************************************************************************************

--- Result 1 ---
Black holes bend spacetime and store immense gravitational energy.

--- Result 2 ---
Python balances readability with power, making it a popular system design language.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 5 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 6 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 7 ---
Consuming leafy greens and fruits helps detox the body and improve 

## ContextualCompressionRetriever

In [46]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [47]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [48]:
# Create a FAISS vector store from the documents
# embedding_model = OpenAIEmbeddings()
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(docs, embedding_model)

In [49]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [51]:
llm = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.5 ## this is creative parameter
    )

In [52]:
# Set up the compressor using an LLM
# llm = ChatOpenAI(model="gpt-3.5-turbo")
compressor = LLMChainExtractor.from_llm(llm)

In [53]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [54]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [55]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.

--- Result 3 ---
Photosynthesis does not occur in animal cells.


In [ ]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
